In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from os import listdir
from os.path import isfile, join

# some comment

embeddings = HuggingFaceEmbeddings(
    model_name = 'emilyalsentzer/Bio_ClinicalBERT'
)
 
text = "This is a test document."
query_result = embeddings.embed_query(text)


No sentence-transformers model found with name emilyalsentzer/Bio_ClinicalBERT. Creating a new one with MEAN pooling.


In [3]:
#onlyfiles = [f for f in listdir('top_1000_txt') if isfile(join('top_1000_txt', f))]
onlyfiles=['note_211.txt']
raw_documents = []
for file in onlyfiles:
    print(file)
    raw_doc = TextLoader(f'top_1000_txt/{file}').load()
    raw_documents.extend(raw_doc)

text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=24)

note_211.txt


In [4]:
documents = text_splitter.split_documents(raw_documents)
db = Chroma.from_documents(documents, embeddings)

In [5]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("johnsnowlabs/JSL-MedLlama-3-8B-v2.0")
model = AutoModelForCausalLM.from_pretrained("johnsnowlabs/JSL-MedLlama-3-8B-v2.0")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:30<00:00, 15.38s/it]


In [6]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA

# 设置 Hugging Face 管道
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=3000,
    truncation=True,
    repetition_penalty=1.15
)

# 创建本地 LLM
local_llm = HuggingFacePipeline(pipeline=pipe)

# 创建检索器
retriever = db.as_retriever(search_kwargs={"k": 1})

# 创建 QA 链
qa_chain = RetrievalQA.from_chain_type(
    llm=local_llm, 
    chain_type="stuff", 
    retriever=retriever, 
    return_source_documents=True
)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [7]:
from operator import itemgetter

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

In [9]:
llm=local_llm
template = """You are an expert clinical assistant. You will receive a collection of clinical notes. You will summarize them in the style of a discharge summary.Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | llm 
    | StrOutputParser()
)

In [10]:
chain.invoke('What allergies did the patient have?')

'Allergic to'

In [11]:
from datasets import Dataset
questions= [
    "Has the patient got a history of diabetes?",
    "What allergies did the patient have?",
    "Does the patient use tobacco or alcohol?",
    "What medication is the patient taking?",
    "What measures were taken for the patient?",
    "What imaging studies were performed on the patient?",
    "What follow-up care was scheduled for the patient?",
    "What did C-spine show?",
    "Does the patient use alcohol?",
    "Does the patient use cigarettes or alcohol?",
    "Did patient use tobacco?"
]

ground_truths = [["No."],
                 ["The patient has no known allergies to drugs."],
                 ["Yes, the patient occasionally uses tobacco (cigarettes) but denies alcohol use."],
                 ["Dilantin 100 IV TID."],
                 ["Intubation and placement of a left radial arterial line for close blood pressure monitoring with a goal of keeping systolic blood pressure under 140. head CT, C-spine, chest/abdomen/pelvis CT to assess injuries. The patient was started on Dilantin 100 IV TID. Monitoring in the Trauma Intensive Care Unit (TICU). Repeat head CT and gradual weaning. Discontinuation of the Foley catheter and intravenous lines.Transfer from the ICU to the general hospital floor.  MRI of the C-spine to evaluate for potential injuries, which showed no fractures or ligamentous injuries, leading to clearance of the C-spine and discontinuation of the C-collar.Evaluation by Neurosurgery and Physical Therapy, deeming the patient fit for discharge with instructions for home care and follow-up."],
                 ["The patient underwent head CT, C-spine, chest/abd/pelvis CT scans, and an MRI of the C-spine"],
                 ["An outpatient CT scan was scheduled to be done in 2 weeks, and a follow-up clinic appointment was scheduled in 2 weeks after the CT scan.No need to follow up in the trauma clinic but the patient may call the clinic with any questions."],
                 ["The MRI of the C-spine showed no fractures or ligamentous injuries."],
                 ["No, the patient denies alcohol use."],
                 ["The patient occasionally uses cigarettes and denies alcohol use."],
                 ["Yes, the patient occasionally uses tobacco (cigarettes)."]

]
answers = []
contexts = []

for query in questions:
    answers.append(chain.invoke(query))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# 构建数据
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}
dataset = Dataset.from_dict(data)


In [12]:
dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truths'],
    num_rows: 11
})

In [13]:
dataset.save_to_disk('JSL-MedLlama-3-8B-v2.0-dataset')

Saving the dataset (1/1 shards): 100%|██████████| 11/11 [00:00<00:00, 618.31 examples/s]


In [14]:
ragas_input_df = dataset.to_pandas()
display(ragas_input_df.head())

,question,answer,contexts,ground_truths
0,Has the patient got a history of diabetes?,"Yes, the patient has a history of diabetes.",[Patient recorded as having No Known Allergies...,[No.]
1,What allergies did the patient have?,Allergic to,[Patient recorded as having No Known Allergies...,[The patient has no known allergies to drugs.]
2,Does the patient use tobacco or alcohol?,Answer: The patient occasionally uses tobacco ...,[Patient recorded as having No Known Allergies...,"[Yes, the patient occasionally uses tobacco (c..."
3,What medication is the patient taking?,The answer is Dilantin.,[Patient recorded as having No Known Allergies...,[Dilantin 100 IV TID.]
4,What measures were taken for the patient?,The answer is: The patient was given Dilantin ...,[Patient recorded as having No Known Allergies...,[Intubation and placement of a left radial art...


In [15]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [18]:
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    context_relevancy,
    answer_correctness,
    answer_similarity
)
     

In [16]:


def evaluate_ragas_dataset(dataset):
  result = evaluate(
    dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
        context_relevancy,
        answer_correctness,
        answer_similarity
    ],
    
  )
  return result

In [19]:
qa_result = evaluate_ragas_dataset(dataset)

evaluating with [context_precision]


100%|██████████| 1/1 [01:02<00:00, 62.34s/it]


evaluating with [faithfulness]


100%|██████████| 1/1 [00:04<00:00,  4.83s/it]


evaluating with [answer_relevancy]


...

100%|██████████| 1/1 [00:02<00:00,  2.83s/it]


evaluating with [answer_correctness]


100%|██████████| 1/1 [00:05<00:00,  5.10s/it]


evaluating with [answer_similarity]


100%|██████████| 1/1 [00:00<00:00,  1.07it/s]


In [20]:
print(qa_result)

{'context_precision': 0.2727, 'faithfulness': 0.7273, 'answer_relevancy': 0.8613, 'context_recall': 1.0000, 'context_relevancy': 0.1193, 'answer_correctness': 0.8750, 'answer_similarity': 1.0000}


In [21]:
df = qa_result.to_pandas()

In [22]:
df.to_csv("JSL-MedLlama-3-8B-v2.0-dataset_ragas_result.csv")